In [1]:
import pandas as pd
import awswrangler as awr
import openpyxl
import os
import shutil
import unicodedata
import win32com.client as win32

#-----------------EXTRACT
path_to_query = r"C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista\listagem_mestra_ajustada.sql"

with open (path_to_query,'r') as reading:
    query_readed = reading.read()

df = awr.athena.read_sql_query(query_readed,'silver')

In [2]:
#-----------------TRANSFORM
#adding empty e-mails with the e-mail common pattern
row = df['e_mail'].isnull() & df['unidade'].str.contains('UNIDADE ')
df.loc[row, 'e_mail'] = 'unidade.' + df.loc[row, 'unidade'].str.replace('UNIDADE ', '').str.lower() + '@transdesk.com.br'


In [3]:
#removing empty spaces
df['e_mail'] = df['e_mail'].str.replace(' ','')

#removing diacritics with unicodedata
def remove_diacritics (texto):
    if isinstance(texto,str):
        return ''.join(c for c in unicodedata.normalize('NFKD',texto) if not unicodedata.combining(c))
    return texto

df['e_mail'] = df['e_mail'].apply(remove_diacritics)

df = df.reset_index(drop=True)


In [4]:
df.head()

,cooperativa,unidade,cliente,matricula,conjunto,coverage,beneficio,motorista,id_placa,placa,...,status,status_coverage,benefits_id,categoria,tipo_categoria,faixa_valor,tabela_ativa,tipo_veiculo,tipo,e_mail
0,Stcoop,UNIDADE ITAJAI,KYELSON KELTE,20898,12597,78174,ANUIDADE,<NA>,<NA>,<NA>,...,ATIVO,ATIVO,39,Anuidade,Anuidade,Anuidade,S,<NA>,RenovaÃ§Ã£o,unidade.itajai@transdesk.com.br
1,Stcoop,UNIDADE CURITIBA,UNYTRANS SERVICOS LOGISTICOS E RODOVIARIOS LTDA,20179,12598,78678,VIDROS,<NA>,12184,SEC0D08,...,ATIVO,ATIVO,31,CAVALO,DE 2024 ATÉ 2020,Cobertura de Vidros,S,Cavalo,RenovaÃ§Ã£o,unidade.curitiba@transdesk.com.br
2,Stcoop,UNIDADE CURITIBA,UNYTRANS SERVICOS LOGISTICOS E RODOVIARIOS LTDA,20179,12598,78674,REPARAÇÃO OU REPOSIÇÃO DO VEÍCULO,<NA>,12184,SEC0D08,...,ATIVO,ATIVO,24,CAVALO,LC: 100% - CP: 6% - FORD E IVECO - 2024 ATÉ 2020,"De R$600.000,01 até R$625.000,00",S,Cavalo,RenovaÃ§Ã£o,unidade.curitiba@transdesk.com.br
3,Stcoop,UNIDADE SÃO PAULO,TSUR TRANSPORTES LTDA,20773,12599,78198,VIDROS,<NA>,12246,SJN8J98,...,ATIVO,ATIVO,31,3 QUARTOS,DE 2024 ATÉ 2020,Cobertura de Vidros,S,Três Quartos,RenovaÃ§Ã£o,
4,Stcoop,UNIDADE SÃO PAULO,TSUR TRANSPORTES LTDA,20773,12599,78197,REPARAÇÃO OU REPOSIÇÃO DO COMPLEMENTO,<NA>,12247,SJN3H80,...,ATIVO,ATIVO,25,TOCO / 3 QUARTOS,TABELA GERAL,"De R$90.000,01 a R$100.000,00",S,Três Quartos,RenovaÃ§Ã£o,


In [39]:
conjuntos_validos = df.loc[df['beneficio']=='SEGURO DE VIDA', 'conjunto'].unique()


df_filtred = df[df['conjunto'].isin(conjuntos_validos)].copy()
df_filtred2 = df_filtred[df_filtred['benefits_id'].isin([2,6,24,28,40,44])].copy()

df_filtred2['casco'] = df_filtred2['beneficio'].apply(lambda x: x if x != 'SEGURO DE VIDA' else '')


df_filtred2['seg_vida'] = df_filtred2['beneficio'].apply(lambda x: x if x == "SEGURO DE VIDA" else '')


df_filtred2 = df_filtred2.groupby(['cooperativa','unidade','matricula','conjunto']).agg(
    qtd_motoristas = ('motorista', 'count'),
    qtd_cascos = ('casco', 'count'),
    qtd_segvida = ('seg_vida', 'count')
).reset_index()



file_path  = r'C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista\conjuntos_analise.xlsx'
df_filtred2.to_excel(file_path, engine = 'openpyxl', index = False)


analisando unidade campo grande

In [11]:
df_beneficios = df.groupby(['unidade','beneficio','faixa_valor']).agg(
    quantidade_placas = ('placa','count'),
    quantidade_motoristas = ('motorista','count')
).reset_index()

df_analise = df_beneficios[
    #(df_beneficios['quantidade_placas']>df_beneficios['quantidade_motoristas'])&
    (df_beneficios['beneficio']=='SEGURO DE VIDA')&
    (df_beneficios['unidade']=='UNIDADE CAMPO GRANDE')&
    (df_beneficios['faixa_valor'].str.contains('MOTORISTA', na = False, case = False))
    ]

display(df_analise)

,unidade,beneficio,faixa_valor,quantidade_placas,quantidade_motoristas
1746,UNIDADE CAMPO GRANDE,SEGURO DE VIDA,MOTORISTA - Seguro de Vida via Rep. ou Rep. do...,0,60
1747,UNIDADE CAMPO GRANDE,SEGURO DE VIDA,MOTORISTA - Seguro de Vida via Rep. ou Rep. do...,0,43


In [19]:
df_analise2 = df[['unidade','beneficio','faixa_valor', 'motorista','placa', 'matricula', 'conjunto', 'coverage','cooperativa']]

df_analise2 = df_analise2[
(df_analise2['beneficio']=='SEGURO DE VIDA')&
(df_analise2['faixa_valor'].str.contains('MOTORISTA', case = False))
]

file_path  = r'C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista\lista_segurodevida.xlsx'
df_analise2.to_excel(file_path, engine = 'openpyxl', index = False)

In [ ]:
df_segurodevida = df_beneficios[df_beneficios['beneficio']=='SEGURO DE VIDA']

file_path = r'C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista\Seguro_de_vida.xlsx'

df_segurodevida.to_excel(file_path, engine='openpyxl', index=False)

In [17]:


file_path = r'C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista\beneficios_geral.xlsx'

df.to_excel(file_path, engine='openpyxl', index=False)

In [9]:
df_unidemail = df.groupby(['unidade', 'e_mail']).agg(
    placas=('placa', 'count'),
    motoristas=('motorista', 'count')
).reset_index()


df_unidemail_filtered = df_unidemail[df_unidemail['placas']>df_unidemail['motoristas']]

df_unidemail_filtered_test = df_unidemail_filtered.head() #ALTEREI PARA TESTE



In [ ]:
#-----------------LOAD
#Function to send e-mails

def enviar_email(p_email, p_unidade, p_placas, p_motoristas):    
    outlook = win32.Dispatch('outlook.application')  #integrating python with outlook 

    #creating e-mail variable to use
    email = outlook.CreateItem(0)

    #configuring the e-mail - ALTEREI PARA TESTE
    email.To = "dados13@grupounus.com.br; dados03@grupounus.com.br"
    email.Subject = "[CADASTRAMENTO DE MOTORISTAS] - Análise de Dados"
    email.HTMLBody = f"""
    <p>Prezado(a),<p/>

    <p>A unidade {p_unidade}, correspondente ao e-mail {p_email}, possui pendências de cadastramento de motoristas por placa:

    <p>Número de placas cadastradas: {p_placas}
    Número de motoristas cadastrados: {p_motoristas}<p/>

    <p>Favor, cadastrar as placas remanescentes para cômputo na base de dados da empresa, obrigado!<p/>

    <p>Atenciosamente,<p/>

    <p>Equipe Análise de Dados - Grupo Unus<p/>

    <p>(<i>Este é um e-mail automático, por favor não responda<i/>)<p/>
 
    """

    email.Send()

    print("Email enviado")

In [ ]:

df_numeros = df_unidemail_filtered.groupby(['unidade']).agg(
    qtd_placas=('placas', 'sum'),
    qtd_motoristas=('motoristas', 'sum')
).reset_index()

for idx, row in df_unidemail_filtered_test.iterrows():
    iEmail = row['e_mail']
    iUnidade = row['unidade']

    unidade_info = df_numeros.loc[df_numeros['unidade'] == iUnidade].iloc[0]

    iQtdPlacas = unidade_info['qtd_placas']
    iQtdMotoristas = unidade_info['qtd_motoristas']

    # if __name__=="__main__":
    #     enviar_email(iEmail, iUnidade, iQtdPlacas, iQtdMotoristas)

    
    print(f"e_mail:ENVIADO PRA GENTE TESTE/unidade:{iUnidade}/placas:{iQtdPlacas}/motoristas:{iQtdMotoristas}")


        

Email enviado
e_mail:ENVIADO PRA GENTE TESTE/unidade:TS CONSULTORIA EM TRANSPORTES LTDA/placas:939/motoristas:218
Email enviado
e_mail:ENVIADO PRA GENTE TESTE/unidade:UNIDADE ARACAJU/placas:14855/motoristas:1558
Email enviado
e_mail:ENVIADO PRA GENTE TESTE/unidade:UNIDADE ARACAJU/placas:14855/motoristas:1558
Email enviado
e_mail:ENVIADO PRA GENTE TESTE/unidade:UNIDADE ARACAJU/placas:14855/motoristas:1558
Email enviado
e_mail:ENVIADO PRA GENTE TESTE/unidade:UNIDADE CAMPO GRANDE/placas:2123/motoristas:110


In [ ]:


path_to_save = r'C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista'
file_name = 'Cobertura de Seguros Motoristas.xlsx'
final_path = os.path.join(path_to_save, file_name)
os.remove(final_path)
df.to_excel(final_path, engine='openpyxl', index = False)